## Aim

I found that when I use strip tags, in five rows, the affiliations will become null. I decided to use BeautifulSoup instead. 

In [93]:
import pandas as pd
import sys
import pandas as pd
import numpy as np
import re
from io import StringIO
from html.parser import HTMLParser
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support as multi_score
from collections import Counter
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [94]:
df = pd.read_csv('../../data/processed/merged_author_df.csv')

In [96]:
df.head(2)

,Year,DOI,Title,IEEE Number of Authors,IEEE Author Position,IEEE Author Name,IEEE Author ID,IEEE Author Affiliation Updated,IEEE One Affiliation,Number of Authors,Author Name,Author Position,Author Position Type,OpenAlex Author ID,Author ORCID,Number of Affiliations,First Institution Name Updated,Raw Affiliation String Updated,First Institution ID,First Institution ROR,First Institution Type Updated,First Institution Country Code Updated,Binary Institution Type,IEEE Author Affiliation Filled
0,2011,10.1109/TVCG.2011.185,D³ Data-Driven Documents,3.0,1.0,Michael Bostock,https://ieeexplore.ieee.org/author/37591067400,"Computer Science Department, Stanford University, Stanford, CA, USA",True,3.0,Michael Bostock,1.0,first,https://openalex.org/A2048345123,NaN,1.0,Stanford University,"Computer Science Department, Stanford University, Stanford, CA, USA#TAB#",https://openalex.org/I97018004,https://ror.org/00f54p054,education,US,education,"Computer Science Department, Stanford University, Stanford, CA, USA"
1,2011,10.1109/TVCG.2011.185,D³ Data-Driven Documents,3.0,2.0,Vadim Ogievetsky,https://ieeexplore.ieee.org/author/38016292400,"Computer Science Department, Stanford University, Stanford, CA, USA",True,3.0,Vadim Ogievetsky,2.0,middle,https://openalex.org/A2668634103,NaN,1.0,Stanford University,"Computer Science Department, Stanford University, Stanford, CA, USA#TAB#",https://openalex.org/I97018004,https://ror.org/00f54p054,education,US,education,"Computer Science Department, Stanford University, Stanford, CA, USA"


In [89]:
df[df['IEEE Author Affiliation Filled'].isnull()]

,Year,DOI,Title,IEEE Number of Authors,IEEE Author Position,IEEE Author Name,IEEE Author ID,IEEE Author Affiliation Updated,IEEE One Affiliation,Number of Authors,Author Name,Author Position,Author Position Type,OpenAlex Author ID,Author ORCID,Number of Affiliations,First Institution Name Updated,Raw Affiliation String Updated,First Institution ID,First Institution ROR,First Institution Type Updated,First Institution Country Code Updated,Binary Institution Type,IEEE Author Affiliation Filled


In [90]:
df[df['IEEE Author Affiliation Filled'].str.contains('&')].shape

(409, 24)

In [91]:
df[df['IEEE Author Affiliation Filled'].str.contains('/')]

,Year,DOI,Title,IEEE Number of Authors,IEEE Author Position,IEEE Author Name,IEEE Author ID,IEEE Author Affiliation Updated,IEEE One Affiliation,Number of Authors,Author Name,Author Position,Author Position Type,OpenAlex Author ID,Author ORCID,Number of Affiliations,First Institution Name Updated,Raw Affiliation String Updated,First Institution ID,First Institution ROR,First Institution Type Updated,First Institution Country Code Updated,Binary Institution Type,IEEE Author Affiliation Filled
145,2004,10.1109/INFVIS.2004.1,A Comparison of the Readability of Graphs Using Node-Link and Matrix-Based Representations,3.0,2.0,J.-D. Fekete,https://ieeexplore.ieee.org/author/37407972900,"INRIA Futurs/LRI, Université Paris Sud, Orsay, France",True,3.0,Jean-Daniel Fekete,2.0,middle,https://openalex.org/A2154968417,https://orcid.org/0000-0003-3770-8726,1.0,French Institute for Research in Computer Science and Automation,INRIA Futurs–LRI,https://openalex.org/I1326498283,https://ror.org/02kvxyf05,government,FR,non-education,"INRIA Futurs/LRI, Université Paris Sud, Orsay, France"
161,1997,10.1109/VISUAL.1997.663888,Application-controlled demand paging for out-of-core visualization,2.0,1.0,M. Cox,https://ieeexplore.ieee.org/author/37089111581,"Microcomputer Research Laboratories, Intel Corporation, MRJ/NASA Ames Research Center, USA",True,2.0,Michael Cox,1.0,first,https://openalex.org/A2298886649,NaN,1.0,Ames Research Center,"MRJ/NASA Ames Research Center, Microcomputer Research Labs, Intel Corporation",https://openalex.org/I1280536761,https://ror.org/02acart68,facility,US,non-education,"Microcomputer Research Laboratories, Intel Corporation, MRJ/NASA Ames Research Center, USA"
162,1997,10.1109/VISUAL.1997.663888,Application-controlled demand paging for out-of-core visualization,2.0,2.0,D. Ellsworth,https://ieeexplore.ieee.org/author/37282594500,"MRJ/NASA Ames Research Center, USA",True,2.0,David S. Ellsworth,2.0,last,https://openalex.org/A2192365028,https://orcid.org/0000-0002-9699-2272,1.0,Ames Research Center,MRJ/NASA Ames Research Center#TAB#,https://openalex.org/I1280536761,https://ror.org/02acart68,facility,US,non-education,"MRJ/NASA Ames Research Center, USA"
165,2004,10.1109/INFVIS.2004.64,The InfoVis Toolkit,1.0,1.0,J.-D. Fekete,https://ieeexplore.ieee.org/author/37407972900,"INRIA Futurs/LRI, Université Paris Sud, Orsay, France",True,1.0,Jean-Daniel Fekete,1.0,first,https://openalex.org/A2154968417,https://orcid.org/0000-0003-3770-8726,1.0,"Univ. de Paris-Sud, Orsay","Univ. de Paris-Sud, Orsay",NaN,NaN,NaN,NaN,NaN,"INRIA Futurs/LRI, Université Paris Sud, Orsay, France"
265,2006,10.1109/TVCG.2006.160,MatrixExplorer: a Dual-Representation System to Explore Social Networks,2.0,1.0,Nathalie Henry,https://ieeexplore.ieee.org/author/37839948300,"INRIA Futurs/LRI and University of Sydney, France",True,2.0,Nathalie Henry,1.0,first,https://openalex.org/A2141074253,NaN,1.0,LRI,LRI,NaN,NaN,NaN,NaN,NaN,"INRIA Futurs/LRI and University of Sydney, France"
266,2006,10.1109/TVCG.2006.160,MatrixExplorer: a Dual-Representation System to Explore Social Networks,2.0,2.0,Jean-daniel Fekete,https://ieeexplore.ieee.org/author/37407972900,"INRIA-Futurs/LRI, France",True,2.0,Jean-Daniel Fekete,2.0,last,https://openalex.org/A2154968417,https://orcid.org/0000-0003-3770-8726,1.0,IEEE Computer Society,[IEEE Computer Society],https://openalex.org/I36253440,https://ror.org/05nxk6n24,nonprofit,US,non-education,"INRIA-Futurs/LRI, France"
326,2002,10.1109/VISUAL.2002.1183757,Interactive rendering of large volume data sets,4.0,1.0,S. Guthe,https://ieeexplore.ieee.org/author/37728224400,"WSI/GRIS, University of Tübingen, Germany",False,4.0,Stefan Guthe,1.0,first,https://openalex.org/A2169808706,https://orcid.org/0000-0001-5539-9096,1.0,University of Tübingen,"WSI/GRIS, Tubingen Univ., Germany",https://openalex.org/I8087733,https://ror.org/03a1kwz48,education,DE,education,"WSI/GRIS, University of Tübingen, Germany"
327,2002,10.1109/VISUAL.2002.1183757,Interactive rende

In [56]:
df[df.DOI == '10.1109/TVCG.2007.70581']

,Year,DOI,Title,IEEE Number of Authors,IEEE Author Position,IEEE Author Name,IEEE Author ID,IEEE Author Affiliation Updated,IEEE One Affiliation,Number of Authors,...,Author ORCID,Number of Affiliations,First Institution Name Updated,Raw Affiliation String Updated,First Institution ID,First Institution ROR,First Institution Type Updated,First Institution Country Code Updated,Binary Institution Type,IEEE Author Affiliation Filled
4566,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,1.0,Randall Smith,https://ieeexplore.ieee.org/author/37280035100,GM R&D,True,5.0,...,NaN,1.0,"GM R&D, Bangalore","GM R&D, Bangalore",NaN,NaN,NaN,NaN,NaN,GM R&D
4567,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,2.0,Richard Pawlicki,https://ieeexplore.ieee.org/author/37282920800,GM R&D,True,5.0,...,NaN,1.0,"GM R&D, Bangalore","GM R&D, Bangalore",NaN,NaN,NaN,NaN,NaN,GM R&D
4568,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,3.0,István Kókai,https://ieeexplore.ieee.org/author/37945249900,"University of Basel, Switzerland",True,5.0,...,NaN,1.0,University of Basel,"#N##TAB##TAB##TAB##TAB# University of Basel, S...",https://openalex.org/I1850255,https://ror.org/02s6k3f65,education,CH,education,"University of Basel, Switzerland"
4569,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,4.0,Jörg Finger,https://ieeexplore.ieee.org/author/37956948500,"University of Basel, Switzerland",True,5.0,...,NaN,1.0,University of Basel,"#N##TAB##TAB##TAB##TAB# University of Basel, S...",https://openalex.org/I1850255,https://ror.org/02s6k3f65,education,CH,education,"University of Basel, Switzerland"
4570,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,5.0,Thomas Vetter,https://ieeexplore.ieee.org/author/37281818300,"University of Basel, Switzerland",True,5.0,...,NaN,1.0,IEEE#TAB#,IEEE#TAB#,NaN,NaN,NaN,NaN,NaN,"University of Basel, Switzerland"


In [101]:
# scrip html tags and entities in titles
# source: https://stackoverflow.com/a/925630
class MLStripper(HTMLParser):
	def __init__(self):
		super().__init__()
		self.reset()
		self.strict = False
		self.convert_charrefs= True
		self.text = StringIO()
	def handle_data(self, d):
		self.text.write(d)
	def get_data(self):
		return self.text.getvalue()

def strip_tags(html):
	s = MLStripper()
	s.feed(html)
	return s.get_data()

def clean_text_old(text):
	"""
	Takes a string and returns a string
	"""
	# lowercase, remove html tags, remove nonsense, remove non-letter
	aff = text.lower()
	aff = strip_tags(aff) 
	aff = re.sub(r'xa0|#n#‡#n#|#tab#|#r#|\[|\]', "", aff)
	aff = re.sub(r'[^a-z]+', ' ', aff)
	return aff

def clean_text(text):
    """
    Takes a string and returns a string
    """
    # remove html tags, lowercase, remove nonsense, remove non-letter
    aff = BeautifulSoup(text, "lxml").text 
    aff = aff.lower()
    aff = re.sub(r'xa0|#n#‡#n#|#tab#|#r#|\[|\]', "", aff)
    aff = re.sub(r'[^a-z]+', ' ', aff)
    return aff

In [99]:
text = '[good]'
text

'[good]'

In [100]:
re.sub(r'[^a-z]+', ' ', text)

' good '

## Try Strip_tags will cause errors

It turns out, if I use `strip_tags`, these authors' affiliations will be rendered null:

- 10.1109/TVCG.2007.70581, author 1, 2
- 10.1109/TVCG.2012.267, author 1, 2, 6

In [69]:
text = 'GM R&D'
# text = 'State Key Lab of CAD&CG, Zhejiang University'
text = text.lower()
text

'gm r&d'

In [70]:
strip_tags(text)

''

In [71]:
text = 'Zhejiang University, State Key Lab of CAD&CG'
text = text.lower()
text

'zhejiang university, state key lab of cad&cg'

In [72]:
strip_tags(text)

''

In [59]:
BeautifulSoup(text, "lxml").text

'gm r&d'

In [60]:
df['IEEE Author Affiliation Filled'] = df['IEEE Author Affiliation Filled'].apply(clean_text)

In [61]:
df[df.DOI == '10.1109/TVCG.2007.70581']

,Year,DOI,Title,IEEE Number of Authors,IEEE Author Position,IEEE Author Name,IEEE Author ID,IEEE Author Affiliation Updated,IEEE One Affiliation,Number of Authors,...,Author ORCID,Number of Affiliations,First Institution Name Updated,Raw Affiliation String Updated,First Institution ID,First Institution ROR,First Institution Type Updated,First Institution Country Code Updated,Binary Institution Type,IEEE Author Affiliation Filled
4566,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,1.0,Randall Smith,https://ieeexplore.ieee.org/author/37280035100,GM R&D,True,5.0,...,NaN,1.0,"GM R&D, Bangalore","GM R&D, Bangalore",NaN,NaN,NaN,NaN,NaN,gm r d
4567,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,2.0,Richard Pawlicki,https://ieeexplore.ieee.org/author/37282920800,GM R&D,True,5.0,...,NaN,1.0,"GM R&D, Bangalore","GM R&D, Bangalore",NaN,NaN,NaN,NaN,NaN,gm r d
4568,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,3.0,István Kókai,https://ieeexplore.ieee.org/author/37945249900,"University of Basel, Switzerland",True,5.0,...,NaN,1.0,University of Basel,"#N##TAB##TAB##TAB##TAB# University of Basel, S...",https://openalex.org/I1850255,https://ror.org/02s6k3f65,education,CH,education,university of basel switzerland
4569,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,4.0,Jörg Finger,https://ieeexplore.ieee.org/author/37956948500,"University of Basel, Switzerland",True,5.0,...,NaN,1.0,University of Basel,"#N##TAB##TAB##TAB##TAB# University of Basel, S...",https://openalex.org/I1850255,https://ror.org/02s6k3f65,education,CH,education,university of basel switzerland
4570,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,5.0,Thomas Vetter,https://ieeexplore.ieee.org/author/37281818300,"University of Basel, Switzerland",True,5.0,...,NaN,1.0,IEEE#TAB#,IEEE#TAB#,NaN,NaN,NaN,NaN,NaN,university of basel switzerland


In [67]:
df[df.DOI == '10.1109/TVCG.2012.267']

,Year,DOI,Title,IEEE Number of Authors,IEEE Author Position,IEEE Author Name,IEEE Author ID,IEEE Author Affiliation Updated,IEEE One Affiliation,Number of Authors,...,Author ORCID,Number of Affiliations,First Institution Name Updated,Raw Affiliation String Updated,First Institution ID,First Institution ROR,First Institution Type Updated,First Institution Country Code Updated,Binary Institution Type,IEEE Author Affiliation Filled
6991,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,1.0,Yubo Tao,https://ieeexplore.ieee.org/author/37631607400,"Zhejiang University, State Key Lab of CAD&CG",True,6.0,...,https://orcid.org/0000-0002-2283-6437,1.0,Zhejiang University,"State Key Lab. of CAD&CG, Zhejiang University,...",https://openalex.org/I76130692,https://ror.org/00a2xv884,education,CN,education,zhejiang university state key lab of cad cg
6992,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,2.0,Hai Lin,https://ieeexplore.ieee.org/author/37085388134,"Zhejiang University, State Key Lab of CAD&CG",True,6.0,...,https://orcid.org/0000-0002-1682-8465,1.0,Zhejiang University,"State Key Lab. of CAD&CG, Zhejiang University,...",https://openalex.org/I76130692,https://ror.org/00a2xv884,education,CN,education,zhejiang university state key lab of cad cg
6993,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,3.0,Feng Dong,https://ieeexplore.ieee.org/author/37436869500,"Visualisation, University of Bedfordshire, UK",True,6.0,...,NaN,1.0,University of Bedfordshire,Centre for Computer Graphics and Visualisation...,https://openalex.org/I147554453,https://ror.org/0400avk24,education,GB,education,visualisation university of bedfordshire uk
6994,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,4.0,Chao Wang,https://ieeexplore.ieee.org/author/37405899600,"Visualisation, University of Bedfordshire, UK",True,6.0,...,NaN,1.0,University of Bedfordshire,Centre for Computer Graphics and Visualisation...,https://openalex.org/I147554453,https://ror.org/0400avk24,education,GB,education,visualisation university of bedfordshire uk
6995,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,5.0,Gordon Clapworthy,https://ieeexplore.ieee.org/author/37283000400,"Visualisation, University of Bedfordshire, UK",True,6.0,...,NaN,1.0,University of Bedfordshire,Centre for Computer Graphics and Visualisation...,https://openalex.org/I147554453,https://ror.org/0400avk24,education,GB,education,visualisation university of bedfordshire uk
6996,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,6.0,Hujun Bao,https://ieeexplore.ieee.org/author/37271755400,"Zhejiang University, State Key Lab of CAD&CG",True,6.0,...,NaN,1.0,Zhejiang University,"State Key Lab. of CAD&CG, Zhejiang University,...",https://openalex.org/I76130692,https://ror.org/00a2xv884,education,CN,education,zhejiang university state key lab of cad cg


In [66]:
df1 = pd.read_csv('../../data/ht_class/merged_country_predicted.csv')
df1[df1['Affiliation Name'].isnull()]

,Year,DOI,Title,Number of Authors,Author Position,Author Name,OpenAlex Author ID,Affiliation Name,Affiliation Country Code
4566,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,1.0,Randall Smith,https://openalex.org/A2156559183,NaN,US
4567,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,2.0,Richard Pawlicki,https://openalex.org/A2078132770,NaN,US
6991,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,1.0,Yubo Tao,https://openalex.org/A2158146757,NaN,US
6992,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,2.0,Hai Lin,https://openalex.org/A2223146649,NaN,US
6996,2012,10.1109/TVCG.2012.267,Structure-Aware Lighting Design for Volume Vis...,6.0,6.0,Hujun Bao,https://openalex.org/A2099886988,NaN,US


In [62]:
df2 = pd.read_csv('../../data/ht_class/merged_aff_type_predicted.csv')
df2[df2['Affiliation Name'].isnull()]

,Year,DOI,Title,Number of Authors,Author Position,Author Name,OpenAlex Author ID,Affiliation Name,Multiclass Affiliation Type,Binary Affiliation Type


In [63]:
df2[df2.DOI == '10.1109/TVCG.2007.70581']

,Year,DOI,Title,Number of Authors,Author Position,Author Name,OpenAlex Author ID,Affiliation Name,Multiclass Affiliation Type,Binary Affiliation Type
4566,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,1.0,Randall Smith,https://openalex.org/A2156559183,gm r d,education,education
4567,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,2.0,Richard Pawlicki,https://openalex.org/A2078132770,gm r d,education,education
4568,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,3.0,István Kókai,https://openalex.org/A2186778383,university of basel switzerland,education,education
4569,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,4.0,Jörg Finger,https://openalex.org/A2422959479,university of basel switzerland,education,education
4570,2007,10.1109/TVCG.2007.70581,Navigating in a Shape Space of Registered Models,5.0,5.0,Thomas Vetter,https://openalex.org/A2065065338,university of basel switzerland,education,education


In [85]:
df2[df2.DOI == '10.1109/INFVIS.2004.1']

,Year,DOI,Title,Number of Authors,Author Position,Author Name,OpenAlex Author ID,Affiliation Name,Multiclass Affiliation Type,Binary Affiliation Type
144,2004,10.1109/INFVIS.2004.1,A Comparison of the Readability of Graphs Using Node-Link and Matrix-Based Representations,3.0,1.0,M. Ghoniem,https://openalex.org/A2699795273,ecole des mines de nantes nantes france,education,education
145,2004,10.1109/INFVIS.2004.1,A Comparison of the Readability of Graphs Using Node-Link and Matrix-Based Representations,3.0,2.0,J.-D. Fekete,https://openalex.org/A2154968417,inria futurs lri universit paris sud orsay france,education,education
146,2004,10.1109/INFVIS.2004.1,A Comparison of the Readability of Graphs Using Node-Link and Matrix-Based Representations,3.0,3.0,P. Castagliola,https://openalex.org/A2053268612,irccyn ecole des mines de nantes nantes france,education,education
